In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from fastdtw import fastdtw
%matplotlib inline

In [2]:
def dateconverter(value): 
    s= str(value)
    Date=datetime.strptime(value, '%m/%d/%y')
    return Date

In [3]:
def get_countydict(df):
    Counties={}
    for value in df.County.unique():
        Counties[value]=df.loc[df['County'] == value]
    return Counties

This section is EDA and cleaning of all drought severity data dealing with California.

In [5]:
CA_SEV_DATA1=pd.read_csv('datasets/California_Drought_Severity_By_County_1997-2002.csv')
CA_SEV_DATA2=pd.read_csv('datasets/California_Drought_Severity_By_County_2002-2007.csv')
CA_SEV_DATA3=pd.read_csv('datasets/California_Drought_Severity_By_County_2007-2012.csv')
CA_SEV_DATA4=pd.read_csv('datasets/California_Drought_Severity_By_County_2012-2017.csv')

FileNotFoundError: File b'project_capstone/part_03/datasets/California_Drought_Severity_By_County_1997-2002.csv' does not exist

In [ ]:
California_Severity_Data=pd.concat([CA_SEV_DATA1,CA_SEV_DATA2,CA_SEV_DATA3,CA_SEV_DATA4],axis=0)
California_Severity_Data.shape

In [ ]:
California_Severity_Data.rename(columns={'Name': 'County'}, inplace=True)

In [ ]:
CA_SEV_Dates=pd.DataFrame(California_Severity_Data.ReleaseDate.apply(dateconverter))

In [ ]:
California_Severity_Data.ReleaseDate=CA_SEV_Dates.ReleaseDate

In [ ]:
CA_years=[]
for value in California_Severity_Data['ReleaseDate']:
    CA_years.append(value.year)

In [ ]:
CAyearsdf=pd.DataFrame(data=CA_years)

In [ ]:
CAyearsdf.rename(columns={0: 'Years'}, inplace=True)

In [ ]:
California_Severity_Data = California_Severity_Data.reset_index(drop=True)

In [ ]:
California_Severity_Data['Year']=CAyearsdf['Years']

In [ ]:
California_Severity_Data.sort_values(by='ReleaseDate',ascending=True,inplace=True)

In [ ]:
California_Severity_Data.drop(['Id','PercentChangeFromWAve',
                             'AreaChangeFromWAve','StatisticFormatID'],axis=1,
                             inplace=True)

In [ ]:
California=get_countydict(California_Severity_Data)

In [ ]:
CABioregiondict={'Sierra County':'Sierra',
               'Amador County':'Sierra',
               'Placer County':'Sierra',
               'Alpine County':'Sierra',
               'Santa Clara County':'Bay/Delta',
               'Yolo County':'Sacramento Valley',
               'Mono County':'Sierra',
               'Ventura County':'Central Coast',
               'Modoc County':'Modoc',
               'Napa County':'Bay/Delta',
               'San Luis Obispo County':'Central Coast',
               'Los Angeles County':'South Coast',
               'Contra Costa County':'Bay/Delta',
               'Yuba County':'Sacramento Valley',
               'Solano County':'Bay/Delta', 
               'Del Norte County':'Klamath/North Coast',
               'Riverside County':'Mojave',
               'Monterey County':'Central Coast',
               'Plumas County':'Sierra',
               'San Francisco County':'Bay/Delta',
               'Mariposa County':'Sierra',
               'Shasta County':'Sacramento Valley',
               'Santa Cruz County':'Central Coast',
               'Tehama County':'Sacramento Valley',
               'Imperial County':'Colorado Desert',
               'Siskiyou County':'Klamath/North Coast',
               'El Dorado County':'Sierra',
               'Mendocino County':'Klamath/North Coast',
               'Fresno County':'San Joaquin Valley',
               'San Bernardino County':'Mojave',
               'San Joaquin County':'San Joaquin Valley',
               'Tulare County':'San Joaquin Valley',
               'Alameda County':'Bay/Delta',
               'Kings County':'San Joaquin Valley',
               'Butte County':'Sacramento Valley',
               'Merced County':'San Joaquin Valley',
               'Sacramento County':'Sacramento Valley',
               'Tuolumne County':'Sierra',
               'Sonoma County':'Bay/Delta',
               'Santa Barbara County':'Central Coast',
               'Madera County':'San Joaquin Valley',
               'Lassen County':'Modoc',
               'Glenn County':'Sacramento Valley',
               'Sutter County':'Sacramento Valley',
               'San Diego County':'South Coast',
               'Lake County':'Klamath/North Coast',
               'Nevada County':'Sierra',
               'Trinity County':'Klamath/North Coast',
               'Inyo County':'Sierra',
               'San Mateo County':'Bay/Delta',
               'Orange County':'South Coast',
               'Humboldt County':'Klamath/North Coast',
               'Colusa County':'Sacramento Valley',
               'Calaveras County':'Sierra',
               'Kern County':'San Joaquin Valley',
               'San Benito County':'Central Coast',
               'Marin County':'Bay/Delta',
               'Stanislaus County':'San Joaquin Valley'
              }
for key, value in CABioregiondict.items():
    California[key]['Bioregion']=value

This section is EDA and cleaning of all drought severity data dealing with Nebraska.

In [ ]:
NE_SEV_DATA1=pd.read_csv('datasets/Nebraska_Drought_Severity_By_County_1997-2002.csv')
NE_SEV_DATA2=pd.read_csv('datasets/Nebraska_Drought_Severity_By_County_2002-2007.csv')
NE_SEV_DATA3=pd.read_csv('datasets/Nebraska_Drought_Severity_By_County_2007-2012.csv')
NE_SEV_DATA4=pd.read_csv('datasets/Nebraska_Drought_Severity_By_County_2012-2017.csv')

In [ ]:
Nebraska_Severity_Data=pd.concat([NE_SEV_DATA1,NE_SEV_DATA2,NE_SEV_DATA3,NE_SEV_DATA4],axis=0)
Nebraska_Severity_Data.shape

In [ ]:
Nebraska_Severity_Data.rename(columns={'Name': 'County'}, inplace=True)

In [ ]:
NE_SEV_Dates=pd.DataFrame(Nebraska_Severity_Data.ReleaseDate.apply(dateconverter))

In [ ]:
Nebraska_Severity_Data.ReleaseDate=NE_SEV_Dates.ReleaseDate

In [ ]:
NE_years=[]
for value in Nebraska_Severity_Data['ReleaseDate']:
   NE_years.append(value.year)

In [ ]:
NEyearsdf=pd.DataFrame(data=NE_years)

In [ ]:
NEyearsdf.rename(columns={0: 'Years'}, inplace=True)

In [ ]:
Nebraska_Severity_Data = Nebraska_Severity_Data.reset_index(drop=True)

In [ ]:
Nebraska_Severity_Data['Year']=NEyearsdf['Years']

In [ ]:
Nebraska_Severity_Data.sort_values(by='ReleaseDate',ascending=True,inplace=True)

In [ ]:
Nebraska_Severity_Data.drop(['Id','PercentChangeFromWAve',
                             'AreaChangeFromWAve','StatisticFormatID'],axis=1,
                             inplace=True)

In [ ]:
Nebraska=get_countydict(Nebraska_Severity_Data)

In [ ]:
NEBioregiondict={'York County':'Tallgrass Prairie',
                 'Cheyenne County':'Shortgrass Prairie',
                 'Rock County':'Sandhills',
                 'Clay County':'Tallgrass Prairie',
                 'Richardson County':'Tallgrass Prairie',
                 'Colfax County':'Tallgrass Prairie',
                 'Red Willow County':'Mixedgrass Prairie',
                 'Cuming County':'Tallgrass Prairie',
                 'Polk County':'Tallgrass Prairie',
                 'Custer County':'Mixedgrass Prairie',
                 'Platte County':'Tallgrass Prairie',
                 'Dakota County':'Tallgrass Prairie',
                 'Pierce County':'Tallgrass Prairie',
                 'Dawes County':'Shortgrass Prairie',
                 'Phelps County':'Mixedgrass Prairie',
                 'Perkins County':'Shortgrass Prairie',
                 'Deuel County':'Shortgrass Prairie',
                 'Pawnee County':'Tallgrass Prairie',
                 'Dixon County':'Tallgrass Prairie',
                 'Otoe County':'Tallgrass Prairie',
                 'Dodge County':'Tallgrass Prairie',
                 'Nuckolls County':'Mixedgrass Prairie',
                 'Douglas County':'Tallgrass Prairie',
                 'Nemaha County':'Tallgrass Prairie',
                 'Dundy County':'Shortgrass Prairie',
                 'Wheeler County':'Sandhills',
                 'Nance County':'Tallgrass Prairie',
                 'Fillmore County':'Tallgrass Prairie',
                 'Saline County':'Tallgrass Prairie',
                 'Morrill County':'Shortgrass Prairie',
                 'Cherry County':'Sandhills',
                 'Chase County':'Shortgrass Prairie',
                 'Antelope County':'Tallgrass Prairie',
                 'Wayne County':'Tallgrass Prairie',
                 'Arthur County':'Sandhills',
                 'Washington County':'Tallgrass Prairie',
                 'Banner County':'Shortgrass Prairie',
                 'Valley County':'Mixedgrass Prairie',
                 'Blaine County':'Sandhills',
                 'Thurston County':'Tallgrass Prairie',
                 'Boone County':'Tallgrass Prairie',
                 'Thomas County':'Sandhills',
                 'Box Butte County':'Shortgrass Prairie',
                 'Thayer County':'Tallgrass Prairie',
                 'Boyd County':'Mixedgrass Prairie',
                 'Stanton County':'Tallgrass Prairie',
                 'Brown County':'Sandhills',
                 'Sioux County':'Shortgrass Prairie',
                 'Buffalo County':'Mixedgrass Prairie',
                 'Sherman County':'Mixedgrass Prairie',
                 'Burt County':'Tallgrass Prairie',
                 'Sheridan County':'Sandhills',
                 'Butler County':'Tallgrass Prairie', 
                 'Adams County':'Mixedgrass Prairie', 
                 'Seward County':'Tallgrass Prairie',
                 'Cass County':'Tallgrass Prairie', 
                 'Scotts Bluff County':'Shortgrass Prairie',
                 'Cedar County':'Tallgrass Prairie',
                 'Saunders County':'Tallgrass Prairie',
                 'Sarpy County':'Tallgrass Prairie', 
                 'Franklin County':'Mixedgrass Prairie',
                 'Dawson County':'Mixedgrass Prairie',
                 'Merrick County':'Mixedgrass Prairie',
                 'Garfield County':'Sandhills',
                 'Lincoln County':'Sandhills',
                 'Gosper County':'Mixedgrass Prairie',
                 'Lancaster County':'Tallgrass Prairie',
                 'Grant County':'Sandhills',
                 'Knox County':'Tallgrass Prairie',
                 'Greeley County':'Mixedgrass Prairie',
                 'Kimball County':'Shortgrass Prairie',
                 'Hall County':'Mixedgrass Prairie',
                 'Keya Paha County':'Mixedgrass Prairie',
                 'Hamilton County':'Tallgrass Prairie', 
                 'Keith County':'Shortgrass Prairie', 
                 'Harlan County':'Mixedgrass Prairie',
                 'Kearney County':'Mixedgrass Prairie',
                 'Hayes County':'Mixedgrass Prairie', 
                 'Johnson County':'Tallgrass Prairie',
                 'Hitchcock County':'Mixedgrass Prairie',
                 'Jefferson County':'Tallgrass Prairie',
                 'Holt County':'Sandhills',
                 'Howard County':'Mixedgrass Prairie',
                 'Hooker County':'Sandhills',
                 'Logan County':'Sandhills',
                 'Garden County':'Sandhills',
                 'Webster County':'Mixedgrass Prairie',
                 'Madison County':'Tallgrass Prairie',
                 'Furnas County':'Mixedgrass Prairie', 
                 'Loup County':'Sandhills',
                 'McPherson County':'Sandhills',
                 'Gage County':'Tallgrass Prairie',
                 'Frontier County':'Mixedgrass Prairie'
                }


for key, value in NEBioregiondict.items():
    Nebraska[key]['Bioregion']=value

In [ ]:
California_Severity_Data.USDMLevel.value_counts()

In [ ]:
California_Severity_Data.shape

In [ ]:
34263/94219*100

In [ ]:
26680/94219*100

In [ ]:
18481/94219*100

In [ ]:
9321/94219*100

In [ ]:
5474/94219*100

In [ ]:
Nebraska_Severity_Data.USDMLevel.value_counts()

In [ ]:
Nebraska_Severity_Data.shape

In [ ]:
51665/117659*100

In [ ]:
32718/117659*100

In [ ]:
20142/117659*100

In [ ]:
9985/117659*100

In [ ]:
3149/117659*100

In [ ]:
List_of_CA_Averages=[]
for keys in California:
    List_of_CA_Averages.append(California[keys].USDMLevelID.mean())

In [ ]:
CA_Avg_List=pd.DataFrame(List_of_CA_Averages)

In [ ]:
CA_Avg_List.rename(columns={0: 'Average'}, inplace=True)

In [ ]:
CA_Avg_List.Average.mean()

In [ ]:
List_of_NE_Averages=[]
for keys in Nebraska:
    List_of_NE_Averages.append(Nebraska[keys].USDMLevelID.mean())

In [ ]:
NE_Avg_List=pd.DataFrame(List_of_NE_Averages)

In [ ]:
NE_Avg_List.rename(columns={0: 'Average'}, inplace=True)

In [ ]:
CA_Bio=pd.DataFrame.from_dict(CABioregiondict,orient='index')

In [ ]:
CA_Bio.rename(columns={0: 'Type'}, inplace=True)

In [ ]:
CA_Bio.index

In [ ]:
NE_Bio=pd.DataFrame.from_dict(NEBioregiondict,orient='index')

In [ ]:
NE_Bio.rename(columns={0: 'Type'}, inplace=True)

In [ ]:
NE_Bio.Type.value_counts()

In [ ]:
CA_Bio.Type.value_counts()

In [ ]:
CA_Bio['County']=CA_Bio.index

In [ ]:
CA_Bio[CA_Bio.Type=='Bay/Delta']

In [ ]:
California['Nevada County'].USDMLevelID.value_counts()

In [ ]:
California['San Bernardino County'].USDMLevelID.value_counts()

In [ ]:
#Make this into a function to use for all the bioregions!
Mojave_List=[]
for key in California:
    if California[key]['Bioregion'].iloc[0]=='Mojave':
        Mojave_List.append(key)
for item in Mojave_List:
    plt.plot(California[item]['Year'],California[item]['USDMLevelID'])

In [ ]:
plt.plot(California['San Bernardino County']['Year'],California['San Bernardino County']['USDMLevelID'])

In [ ]:
plt.plot(Nebraska['Garden County']['Year'],Nebraska['Garden County']['USDMLevelID'])

In [ ]:
#for key in Nebraska:
    #print(Nebraska[key].shape)

In [ ]:
CA_agro_counties=['Fresno County',
'Tulare County',
'Monterey County',
'Kern County',
'Merced County',
'Stanislaus County',
'San Joaquin County',
'Ventura County',
'San Diego County',
'Imperial County']
len(CA_agro_counties)

In [ ]:
DTWvalues={}
for value in California.keys_!='Fresno County'():
    test_list=fastdtw(California[value].USDMLevelID,
        California['Fresno County'].USDMLevelID)
    DTWvalues[value]=test_list[0]
    #test_list2=test_list[1:]
   # x = []; y=[]
    #for point in test_list2[0]:
       # x.append(point[0])
        #y.append(point[1])
    #plt.figure()
   # plt.scatter(x,y)

In [ ]:
for value in CA_agro_counties:
    print(DTWvalues[value])

In [ ]:
DTWvalues2={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['Tulare County'].USDMLevelID)
    DTWvalues2[value]=test_list[0]

In [ ]:
DTWvalues3={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['Monterey County'].USDMLevelID)
    DTWvalues3[value]=test_list[0]

'Merced County',
'Stanislaus County',
'San Joaquin County',
'Ventura County',
'',
'Imperial County'

In [ ]:
#test_list2

In [ ]:
DTWvalues4={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['Kern County'].USDMLevelID)
    DTWvalues4[value]=test_list[0]

In [ ]:
DTWvalues5={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['Merced County'].USDMLevelID)
    DTWvalues5[value]=test_list[0]

In [ ]:
DTWvalues6={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['Stanislaus County'].USDMLevelID)
    DTWvalues6[value]=test_list[0]
    

In [ ]:
DTWvalues7={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['San Joaquin County'].USDMLevelID)
    DTWvalues7[value]=test_list[0]

In [ ]:
DTWvalues8={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['Ventura County'].USDMLevelID)
    DTWvalues8[value]=test_list[0]

In [ ]:
DTWvalues9={}
for value in California.keys():
    test_list=fastdtw(California[value].USDMLevelID,
        California['San Diego County'].USDMLevelID)
    DTWvalues9[value]=test_list[0]

In [ ]:
print

In [ ]:
test_list3=fastdtw(California['Monterey County'].USDMLevelID,California['Fresno County'].USDMLevelID)

In [ ]:
max(DTWvalues, key=DTWvalues.get)

In [ ]:
max(DTWvalues2, key=DTWvalues2.get)

In [ ]:
max(DTWvalues3, key=DTWvalues3.get)

In [ ]:
max(DTWvalues4, key=DTWvalues4.get)

In [ ]:
max(DTWvalues5, key=DTWvalues5.get)

In [ ]:
max(DTWvalues6, key=DTWvalues6.get)

In [ ]:
max(DTWvalues7, key=DTWvalues7.get)

In [ ]:
max(DTWvalues8, key=DTWvalues8.get)

In [ ]:
max(DTWvalues9, key=DTWvalues9.get)